In [1]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
import os
import time

# Tạo thư mục lưu ảnh và video
output_img_dir = 'output_images'
output_video_dir = 'output_videos'
output_time_file = 'time_log.txt'  # File lưu thời gian in-out
output_img_visulize = 'output_visualize'
video_path = "Pickleball Thủ Đô TV Live Stream - Pickleball Thủ Đô TV (720p, h264).mp4"
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_video_dir, exist_ok=True)
os.makedirs(output_img_visulize, exist_ok=True)


# Đặt tên cho video highlight
court_points = []  # Danh sách lưu các điểm góc sân

# Hàm xử lý sự kiện khi người dùng click chọn các góc sân
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        court_points.append((x, y))
        print(f"Point {len(court_points)}: ({x}, {y})")

# Load video và chọn điểm góc sân

cap = cv2.VideoCapture(video_path)
success, frame = cap.read()

if not success:
    print("Không thể đọc frame đầu tiên!")
    exit()

cv2.imshow("Select 4 Court Corners", frame)
cv2.setMouseCallback("Select 4 Court Corners", click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

if len(court_points) != 4:
    print("Cần chọn đúng 4 điểm!")
    exit()

# Vẽ các đường nối giữa 4 điểm góc sân
for i in range(4):
    pt1 = court_points[i]
    pt2 = court_points[(i + 1) % 4]
    cv2.line(frame, pt1, pt2, (255, 0, 0), 2)
cv2.imshow("Court Lines", frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

print(court_points)


src_pts = np.float32(court_points)
dst_pts = np.float32([[0, 0], [300, 0], [300, 600], [0, 600]])
H = cv2.getPerspectiveTransform(src_pts, dst_pts)

# Hàm hỗ trợ biến đổi điểm (warp) từ khung gốc sang sân
def warp_point(pt, H):
    pt_h = np.array([[pt]], dtype='float32')
    pt_warped = cv2.perspectiveTransform(pt_h, H)
    return tuple(pt_warped[0][0])

# Hàm kiểm tra bóng có ở trong sân hay ngoài sân
def check_in_out(pt, w=300, h=600):
    x, y = pt
    return "IN" if 0 <= x <= w and 0 <= y <= h else "OUT"

# Hàm kiểm tra khi nào bóng chạm đất
def is_landing(track, i, threshold=15):
    if i < 1 or i+1 >= len(track):
        return False
    d1 = np.linalg.norm(np.array(track[i]) - np.array(track[i-1]))
    d2 = np.linalg.norm(np.array(track[i+1]) - np.array(track[i]))
    return d1 > threshold and d2 < threshold

# Hàm vẽ sân tennis
def draw_tennis_court(canvas, offset_x=100, offset_y=100, width=300, height=600):
    court_color = (0, 0, 0)
    line_thickness = 2

    # Outer court
    cv2.rectangle(canvas, (offset_x, offset_y), (offset_x + width, offset_y + height), court_color, line_thickness)

    # Midline (baseline giữa sân)
    cv2.line(canvas,
             (offset_x, offset_y + height // 2),
             (offset_x + width, offset_y + height // 2),
             court_color, line_thickness)

    # Service boxes (chia 4 vùng giao bóng)
    cv2.line(canvas,
             (offset_x + width // 2, offset_y),
             (offset_x + width // 2, offset_y + height),
             court_color, line_thickness)

    return canvas

# Load YOLO model
model = YOLO("best_ball_yolo12n.pt")
cap = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: [])

court_canvas = np.ones((800, 500, 3), dtype=np.uint8) * 255

# Tạo VideoWriter để lưu video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video_path = os.path.join(output_video_dir, 'full_video_output.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(out_video_path, fourcc, fps, (frame_width, frame_height))

# Mở file để lưu thời gian in-out
with open(output_time_file, 'w') as time_file:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Ghi video gốc vào file
        out.write(frame)

        # Vẽ lại sân trên canvas phụ mỗi vòng lặp (xóa các điểm cũ)
        court_canvas = np.ones((800, 500, 3), dtype=np.uint8) * 255
        court_canvas = draw_tennis_court(court_canvas)

        # Sử dụng YOLOv8 để theo dõi quả bóng
        result = model.track(frame, persist=True, verbose=False, conf=0.3)[0]
        if result.boxes and result.boxes.id is not None:
            boxes = result.boxes.xywh.cpu()
            track_ids = [1 for _ in boxes]

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 30:
                    track.pop(0)

                # Vẽ đường đi của bóng
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

            # Phát hiện điểm chạm đất và kiểm tra in/out
            for track_id, track in track_history.items():
                if len(track) >= 3:
                    for i in range(1, len(track)-1):
                        if is_landing(track, i):
                            landing_pt = track[i]
                            court_pt = warp_point(landing_pt, H)
                            status = check_in_out(court_pt)
                            cx, cy = map(int, court_pt)
                            canvas_x = cx + 100
                            canvas_y = cy + 100
                            color = (0, 255, 0) if status == "IN" else (0, 0, 255)
                            cv2.circle(court_canvas, (canvas_x, canvas_y), 10, color, -1)
                            cv2.putText(court_canvas, status, (canvas_x + 10, canvas_y - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                            # Ghi lại thời gian in/out vào file
                            timestamp = time.strftime("%H:%M:%S", time.gmtime(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000))
                            time_file.write(f"{status};{timestamp};{cx};{cy}\n")

                            # Capture lại những lần IN và OUT vào ảnh
                            img_filename_visualize = os.path.join(output_img_visulize, f"{status}_{timestamp}.png")
                            cv2.imwrite(img_filename_visualize, court_canvas)
                            img_filename = os.path.join(output_img_dir, f"{status}_{timestamp}.png")
                            cv2.imwrite(img_filename, frame)

        # Hiển thị frame video gốc và canvas sân
        cv2.imshow("YOLOv8 Tennis Tracking", frame)
        cv2.imshow("Tennis Court with Ball Tracking", court_canvas)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

# Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()


Point 1: (482, 202)
Point 2: (808, 197)
Point 3: (1098, 650)
Point 4: (171, 656)
[(482, 202), (808, 197), (1098, 650), (171, 656)]


In [1]:
import socket  # thêm ở đầu file nếu chưa có

if __name__ == "__main__":
    try:
        # Cách chính xác để lấy local IP (tránh localhost 127.0.0.1)
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            # Kết nối tạm đến một địa chỉ ngoài để hệ thống chọn đúng IP nội bộ
            s.connect(("8.8.8.8", 80))
            local_ip = s.getsockname()[0]
        finally:
            s.close()

        print(f"📡 Server is running at: http://{local_ip}:5000")
        print("🚀 Starting server on port 5000...")

        socketio.run(app, host="0.0.0.0", port=5000)
    finally:
        if out is not None:
            with video_lock:
                out.release()
        if cap.isOpened():
            with video_lock:
                cap.release()


📡 Server is running at: http://192.168.1.8:5000
🚀 Starting server on port 5000...


NameError: name 'out' is not defined